# Decision Tree

In [18]:
# Load our libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Use a nicer style for plots
plt.style.use("seaborn-v0_8-muted")

# Import the regression tree from scikit-learn and a plotting helper
from sklearn.tree import DecisionTreeRegressor, plot_tree
# Import our train_test_split helper
from sklearn.model_selection import train_test_split

## Prepare Dataset into target and features and split them into test train Validation

In [19]:
target_data = pd.read_csv("../../../4 - Data/04_WorkingDatasets/02 NormalCombLagged/TargetOutliersTreated.csv")


features_data = pd.read_csv("../../../4 - Data/04_WorkingDatasets/02 NormalCombLagged/WDFCombLagged.csv")

In [20]:
# Konvertiere die Datumsspalte in einen datetime-Index (falls nicht bereits)
#target_data['Datum'] = pd.to_datetime(target_data['Datum'])
#features_data['Datum'] = pd.to_datetime(features_data['Datum'])

# Definiere das Cut-Off-Datum
#cutoff_date = pd.Timestamp('2024-10-20 21:00:00+00:00')

# Filtere das Dataset auf Einträge bis einschließlich des Cut-Off-Datums
#target_data_cutted = target_data[target_data['Datum'] <= cutoff_date]
#features_data_cutted = features_data[features_data['Datum'] <= cutoff_date]


features_data = features_data[features_data['Datum'] >= '2022-01-02 01:00:00+00:00']

target_data = target_data[target_data['Datum'] <= '2024-10-19 20:00:00+00:00']



In [21]:
# Split our data intro features and targets
# Teile das Dataset in Features und Zielvariable
y = target_data["PM10_Combined_Trend_Residual"]  # Zielvariable
X = features_data.drop(columns=["Datum"])  # Alle Spalten außer der Zielvariable

X.head(10)

# Daten splitten
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=11)  # 10% von Gesamt

print("Train Size:", len(X_train))
print("Validation Size:", len(X_val))
print("Test Size:", len(X_test))

Train Size: 17166
Validation Size: 2453
Test Size: 4905


## Decision Tree Regressor

In [22]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(11)

# Expanding Cross-Validation (5 Splits)
tscv = TimeSeriesSplit(n_splits=5)
results = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    # Expanding training data with each fold
    X_train_fold = X_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    X_val_fold = X_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    model = DecisionTreeRegressor()
    model.fit(X_train_fold, y_train_fold)
    
    # Predict and calculate MSE
    y_pred = model.predict(X_val_fold)
    mse = mean_squared_error(y_val_fold, y_pred)
    results.append(mse)
    print(f"Fold {fold + 1}: MSE = {mse}")

# Average MSE across all folds
average_mse = np.mean(results)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse}")

Fold 1: MSE = 12.988289686224334
Fold 2: MSE = 10.780417168043064
Fold 3: MSE = 12.060494458925092
Fold 4: MSE = 13.300713682582808
Fold 5: MSE = 9.319648158934031
Durchschnittlicher MSE über alle Folds: 11.689912630941866


## Use ccp_alpha (and other parameters) to optimize the Decision Tree for example when it comes to overfitting

[`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [23]:
np.random.seed(11)# letter K in Alphabet

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
results_ccp = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    # Expanding training data with each fold
    X_train_fold = X_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    X_val_fold = X_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Modell trainieren
    tree_ccp = DecisionTreeRegressor(ccp_alpha=0.01)
    tree_ccp.fit(X_train_fold, y_train_fold)
    
    # Vorhersagen machen
    y_pred_ccp = tree_ccp.predict(X_val_fold)
    
    # Berechne den Fehler
    mse_ccp = mean_squared_error(y_val_fold, y_pred_ccp)
    results_ccp.append(mse_ccp)
    
    print(f"Fold {fold + 1}: MSE = {mse_ccp}")

# Durchschnittlichen Fehler über alle Folds berechnen
average_mse_ccp = np.mean(results_ccp)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse_ccp}")


Fold 1: MSE = 11.706150615310897
Fold 2: MSE = 8.324985502071323
Fold 3: MSE = 9.037343305875543
Fold 4: MSE = 10.277567752247782
Fold 5: MSE = 5.812763439806636
Durchschnittlicher MSE über alle Folds: 9.031762123062437


## Crossvalidation to find best alpha 

In [24]:
# Import our tools for model selection
from sklearn.model_selection import cross_validate, KFold

In [25]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor
import numpy as np

np.random.seed(11)

# Definiere die Werte für ccp_alpha (Cost Complexity Pruning)
alphas = np.logspace(-4, 0, 50)  # Werte zwischen 10^-4 und 10^0

# Initialisiere Cross-Validation
cv = KFold(n_splits=5, shuffle=True, random_state=11)

# Speicher für Ergebnisse
scores = []  # Durchschnittliche MSE für jedes ccp_alpha
scores_std = []  # Standardabweichung der Scores für Stabilitätsanalyse

# Cross-Validation für jedes ccp_alpha
for alpha in alphas:
    # Decision Tree mit aktuellem ccp_alpha-Wert
    tree_cv = DecisionTreeRegressor(ccp_alpha=alpha, random_state=11)
    
    # Negative MSE, da cross_val_score maximiert; wir wollen minimieren
    mse_scores = cross_val_score(tree_cv, X_train, y_train, cv=cv, scoring="neg_mean_squared_error")
    
    # Durchschnittlichen MSE speichern (negativ, daher multiplizieren mit -1)
    scores.append(-mse_scores.mean())
    scores_std.append(mse_scores.std())

# Optimiere ccp_alpha: Der Wert mit dem niedrigsten MSE
optimal_alpha = alphas[np.argmin(scores)]

print(f"Optimales ccp_alpha: {optimal_alpha}")

Optimales ccp_alpha: 0.10481131341546852


In [26]:
np.random.seed(11)

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
results_ccp = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    # Expanding training data with each fold
    X_train_fold = X_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    X_val_fold = X_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Modell trainieren
    tree_ccp = DecisionTreeRegressor(ccp_alpha=optimal_alpha)
    tree_ccp.fit(X_train_fold, y_train_fold)
    
    # Vorhersagen machen
    y_pred_ccp = tree_ccp.predict(X_val_fold)
    
    # Berechne den Fehler
    mse_ccp = mean_squared_error(y_val_fold, y_pred_ccp)
    results_ccp.append(mse_ccp)
    
    print(f"Fold {fold + 1}: MSE = {mse_ccp}")

# Durchschnittlichen Fehler über alle Folds berechnen
average_mse_ccp = np.mean(results_ccp)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse_ccp}")


Fold 1: MSE = 9.762933126265777
Fold 2: MSE = 7.6768667659060155
Fold 3: MSE = 6.862300211764157
Fold 4: MSE = 7.868051517151952
Fold 5: MSE = 5.208352253861845
Durchschnittlicher MSE über alle Folds: 7.47570077498995


## Bagging

Bagging (Bootstrap Aggregating) ist eine Technik, um die Stabilität und Genauigkeit von Machine-Learning-Algorithmen zu verbessern, insbesondere bei Modellen wie Entscheidungsbäumen, die anfällig für hohe Varianz sind. Es basiert auf dem Bootstrapping-Prinzip, bei dem mehrere Trainingssets durch Zufallsstichproben mit Zurücklegen erzeugt werden.

Jeder Baum wird auf einem dieser zufälligen Datensets trainiert, und die Vorhersagen der B Modelle werden durch Mittelung kombiniert. Mathematisch reduziert Bagging die Varianz der Modelle, weil unabhängige Fehler über die Modelle hinweg geglättet werden. So wird die Vorhersage insgesamt stabiler und robuster gegen Variationen in den Trainingsdaten.

Das Ziel ist, Vorhersagefehler durch Mittelung der Outputs der individuellen Modelle zu minimieren, was insgesamt zu einer besseren Modellleistung führt.

In [27]:
# Import the regression tree from scikit-learn and a plotting helper
from sklearn.tree import DecisionTreeRegressor, plot_tree
# Import our train_test_split helper
from sklearn.model_selection import train_test_split
# Import the mean_squared_error function under the alias mse
from sklearn.metrics import mean_squared_error as mse
# Import the resampling helper
from sklearn.utils import resample
# Import the sklearn implementation of bagging
from sklearn.ensemble import BaggingRegressor

np.random.seed(11)



# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
results_bag = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    # Expanding training data with each fold
    X_train_fold = X_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    X_val_fold = X_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]

    bagged_trees = BaggingRegressor(DecisionTreeRegressor(), n_estimators=100)
    
    # Modell trainieren
    bagged_trees.fit(X_train_fold, y_train_fold)
    
    # Vorhersagen machen
    y_pred = bagged_trees.predict(X_val_fold)
    
    # Berechne den Fehler
    mse = mean_squared_error(y_val_fold, y_pred)
    results_bag.append(mse)
    
    print(f"Fold {fold + 1}: MSE = {mse}")

# Durchschnittlichen Fehler über alle Folds berechnen
average_mse_bag = np.mean(results_bag)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse_bag}")


Fold 1: MSE = 7.843239355054653
Fold 2: MSE = 6.152337930307833
Fold 3: MSE = 5.294114413792775
Fold 4: MSE = 8.365999843434036
Fold 5: MSE = 4.663225097904532
Durchschnittlicher MSE über alle Folds: 6.463783328098766


## Random Forrest

Random Forests erweitern Bagging, indem sie jedem Baum eine zusätzliche Zufallskomponente hinzufügen. Jeder Baum wird mit einem bootstrap-Sample der Trainingsdaten trainiert, wobei nur ein zufälliger Teil der Features für die Konstruktion des Baums verwendet wird. Dadurch unterscheidet sich Random Forests von klassischem Bagging, bei dem alle Features verfügbar sind.

Die zufällige Auswahl der Features reduziert die Korrelation zwischen den Bäumen und verbessert die Generalisierung des Modells. Üblicherweise wird die Anzahl der verwendeten Features  m  so gewählt, dass  m \approx \sqrt{p} , wobei  p  die Gesamtzahl der Features ist. Wenn  m = p  gesetzt wird, ist Random Forest gleichbedeutend mit einem Bagging-Modell.

In [28]:
# Import the random forest regressor
from sklearn.ensemble import RandomForestRegressor

np.random.seed(11)

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
results_rf = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    X_train_fold, X_val_fold = X_train_init.iloc[train_index], X_train_init.iloc[val_index]
    y_train_fold, y_val_fold = y_train_init.iloc[train_index], y_train_init.iloc[val_index]

    # Initialize the random forest regressor
    rf = RandomForestRegressor(n_estimators=100, max_features="sqrt")
    
    # Modell trainieren
    rf.fit(X_train_fold, y_train_fold)
    
    # Vorhersagen machen
    y_pred = rf.predict(X_val_fold)
    
    # Berechne den Fehler
    fold_mse = mean_squared_error(y_val_fold, y_pred)
    results_rf.append(fold_mse)
    
    print(f"Fold {fold + 1}: MSE = {fold_mse}")

# Durchschnittlichen Fehler über alle Folds berechnen
average_mse_rf = np.mean(results_rf)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse_rf}")

Fold 1: MSE = 17.295614660975417
Fold 2: MSE = 11.197622019781285
Fold 3: MSE = 9.686280765736692
Fold 4: MSE = 15.208287529040764
Fold 5: MSE = 7.533474033293184
Durchschnittlicher MSE über alle Folds: 12.184255801765467


Der Unterschied zwischen der Verwendung eines Integers oder eines Floats bei der Angabe von max_features in einem Random Forest Modell (wie in Scikit-learn) liegt in der Bedeutung des Parameters und wie die Anzahl der maximal zu betrachtenden Features berechnet wird:

1. Wenn max_features ein Integer ist:

	•	Der Wert gibt die exakte Anzahl der maximal zu betrachtenden Features an, die bei der Teilung eines Knotens in jedem Decision Tree berücksichtigt werden sollen.
	•	Beispiel: max_features=3 bedeutet, dass 3 Features aus dem gesamten Feature-Set zufällig ausgewählt werden, um die beste Teilung zu bestimmen.

2. Wenn max_features ein Float ist:

	•	Der Wert gibt einen Prozentsatz der verfügbaren Features an, die verwendet werden sollen. Der Float-Wert muss zwischen 0.0 und 1.0 liegen.
	•	Beispiel: max_features=0.5 bedeutet, dass 50 % der Features (aufgerundet) zufällig ausgewählt werden, um die beste Teilung zu bestimmen.
    

# Lagged Variable mit 1h lag

In [29]:
# Ensure 'Datum' column is in datetime format
#target_data_cutted['Datum'] = pd.to_datetime(target_data_cutted['Datum']).copy()

# Create a copy of target_data to apply the offset
lagged_target_variable_1h = target_data.copy()


# Offset von -1 Stunde anwenden
#lagged_target_variable_1h['Datum'] = target_data['Datum'] + pd.Timedelta(hours=-1)

lagged_target_variable_1h.head()


,Datum,PM10_Combined_Trend_Residual
0,2022-01-02 01:00:00+00:00,14.547155
1,2022-01-02 02:00:00+00:00,13.151571
2,2022-01-02 03:00:00+00:00,12.394376
3,2022-01-02 04:00:00+00:00,11.062399
4,2022-01-02 05:00:00+00:00,11.068747


# Ensure 'Datum' column in features_data is in datetime format
features_data_cutted['Datum'] = pd.to_datetime(features_data['Datum'])

# Merge the dataframes
features_data_lagged = pd.merge(features_data_cutted, lagged_target_variable_1h, on="Datum", how="left")

features_data_lagged.head(5)

# Models mit Lagged Value trainieren

In [32]:
# Split our data intro features and targets
# Teile das Dataset in Features und Zielvariable
y = target_data["PM10_Combined_Trend_Residual"]  # Zielvariable
L = features_data.drop(columns=["Datum"])  # Alle Spalten außer der Zielvariable

X.head(10)

# Split into training and test sets
#L_train, L_test, y_train, y_test = train_test_split(L, y, random_state=72)

# Daten splitten
L_train_val, L_test, y_train_val, y_test = train_test_split(L, y, test_size=0.2, random_state=11)
L_train, L_val, y_train, y_val = train_test_split(L_train_val, y_train_val, test_size=0.125, random_state=11)  # 10% von Gesamt

print("Train Size:", len(L_train))
print("Validation Size:", len(L_val))
print("Test Size:", len(L_test))

Train Size: 17166
Validation Size: 2453
Test Size: 4905


## Normaler Decision Tree

In [33]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(11)

# Expanding Cross-Validation (5 Splits)
tscv = TimeSeriesSplit(n_splits=5)
L_results = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
L_train_init = L_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(L_train_init)):
    # Expanding training data with each fold
    L_train_fold = L_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    L_val_fold = L_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    L_model = DecisionTreeRegressor()
    L_model.fit(L_train_fold, y_train_fold)
    
    # Predict and calculate MSE
    L_y_pred = L_model.predict(L_val_fold)
    L_mse = mean_squared_error(y_val_fold, L_y_pred)
    L_results.append(L_mse)
    print(f"Fold {fold + 1}: MSE = {L_mse}")

# Average MSE across all folds
L_average_mse = np.mean(L_results)
print(f"Durchschnittlicher MSE über alle Folds: {L_average_mse}")

Fold 1: MSE = 12.988289686224334
Fold 2: MSE = 10.780417168043064
Fold 3: MSE = 12.060494458925092
Fold 4: MSE = 13.300713682582808
Fold 5: MSE = 9.319648158934031
Durchschnittlicher MSE über alle Folds: 11.689912630941866


## Decision Tree mit Optimierung ccp_alpha

In [34]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

np.random.seed(11)

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
L_results_ccp = []

for fold, (train_index, val_index) in enumerate(tscv.split(L_train_init)):
    # Expanding training data with each fold
    L_train_fold = L_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    L_val_fold = L_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    L_tree_ccp = DecisionTreeRegressor(ccp_alpha=0.01)
    L_tree_ccp.fit(L_train_fold, y_train_fold)
    
    # Vorhersagen machen
    L_y_pred_ccp = L_tree_ccp.predict(L_test)
    
    # Berechne den Fehler
    L_mse_ccp = mean_squared_error(y_test, L_y_pred_ccp)
    L_results_ccp.append(L_mse_ccp)
    
    print(f"Fold {fold + 1}: MSE = {L_mse_ccp}")

# Durchschnittlichen Fehler über alle Folds berechnen
L_average_mse_ccp = np.mean(L_results_ccp)
print(f"Durchschnittlicher MSE über alle Folds: {L_average_mse_ccp}")

Fold 1: MSE = 12.512141241857044
Fold 2: MSE = 12.478191820788073
Fold 3: MSE = 11.403309277779476
Fold 4: MSE = 12.338960178672899
Fold 5: MSE = 12.095796584200055
Durchschnittlicher MSE über alle Folds: 12.16567982065951


## Bagging

In [35]:
# Import the regression tree from scikit-learn and a plotting helper
from sklearn.tree import DecisionTreeRegressor, plot_tree
# Import our train_test_split helper
from sklearn.model_selection import train_test_split
# Import the mean_squared_error function under the alias mse
from sklearn.metrics import mean_squared_error as mse
# Import the resampling helper
from sklearn.utils import resample
# Import the sklearn implementation of bagging
from sklearn.ensemble import BaggingRegressor

np.random.seed(11)

B = 100


# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
L_results_bag = []


for fold, (train_index, val_index) in enumerate(tscv.split(L_train_init)):
    # Expanding training data with each fold
    L_train_fold = L_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    L_val_fold = L_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    L_bagged_trees = BaggingRegressor(DecisionTreeRegressor(), n_estimators=B, random_state=11)
    L_bagged_trees.fit(L_train_fold, y_train_fold)
    
    # Vorhersagen machen
    L_y_pred = L_bagged_trees.predict(L_test)
    
    # Berechne den Fehler
    L_mse = mean_squared_error(y_test, L_y_pred)
    L_results_bag.append(L_mse)
    
    print(f"Fold {fold + 1}: MSE = {L_mse}")

# Durchschnittlichen Fehler über alle Folds berechnen
L_average_mse_bag = np.mean(L_results_bag)
print(f"Durchschnittlicher MSE über alle Folds: {L_average_mse_bag}")

Fold 1: MSE = 8.933200460251955
Fold 2: MSE = 8.377148684955634
Fold 3: MSE = 8.64273747118487
Fold 4: MSE = 8.652191076258385
Fold 5: MSE = 9.251124441030038
Durchschnittlicher MSE über alle Folds: 8.771280426736176


## Random Forest

In [36]:
# Import the random forest regressor
from sklearn.ensemble import RandomForestRegressor

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
L_results_rf = []

for fold, (train_index, val_index) in enumerate(tscv.split(L_train_init)):
    # Expanding training data with each fold
    L_train_fold = L_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    L_val_fold = L_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    L_rf = RandomForestRegressor(n_estimators=B, max_features="sqrt", random_state=11)
    L_rf.fit(L_train_fold, y_train_fold)

    # Vorhersagen machen
    L_y_pred = L_rf.predict(L_test)
    
    # Berechne den Fehler
    L_fold_mse = mean_squared_error(y_test, L_y_pred)
    L_results_rf.append(L_fold_mse)
    
    print(f"Fold {fold + 1}: MSE = {L_fold_mse}")

# Durchschnittlichen Fehler über alle Folds berechnen
L_average_mse_rf = np.mean(L_results_rf)
print(f"Durchschnittlicher MSE über alle Folds: {L_average_mse_rf}")

Fold 1: MSE = 15.87225525103314
Fold 2: MSE = 14.618220201524531
Fold 3: MSE = 13.906914792305509
Fold 4: MSE = 13.60908511624768
Fold 5: MSE = 13.337918745966594
Durchschnittlicher MSE über alle Folds: 14.26887882141549
